# Chapter 5: Resampling Methods

Resampling methods are an indispensable tool in modern statistics. They involve repeatedly drawing samples from a training set and refitting a model of interest on each sample in order to obtain additional information about the fitted model.  Two of the most commonly used resampling methods are cross-validation and bootstrap. 

- Cross-validation can be used to estimate the test error associated with a given statistical learning method in order to evaluate its performance, or to select the appropriate level of flexibility. The process of evaluating a model’s performance is known as model assessment, whereas the process of selecting the proper level of flexibility for a model is known as model selection. 

- Bootstrap is used in several contexts, most commonly to provide a measure of accuracy of a parameter estimate or of a given statistical learning method.


# 5.1. Cross Validation

In general, the training error rate often is quite different from the test error rate, and in particular the former can dramatically underestimate the latter.

In the absence of a very large designated test set that can be used to directly estimate the test error rate, a number of techniques can be used to estimate this quantity using the available training data. Some methods make a mathematical adjustment to the training error rate in order to estimate the test error rate.

In this section, we instead consider a class of methods that estimate the test error rate by holding out a subset of the training observations from the fitting process, and then applying the statistical learning method to those held out observations. For simplicity we assume that we are interested in performing regression with a quantitative response. Next, we consider the case of classification with a qualitative response.

# The (single) validation set approach


We begin by using the sample() function to split the set of observations sample() into two halves, by selecting a random subset of 196 observations out of
the original 392 observations.

In [1]:
library(ISLR2)
attach(Auto)

In [2]:
dim(Auto)

[1] 392   9

In [3]:
head(Auto)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,<dbl>,<int>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<fct>
1,18,8,307,130,3504,12.0,70,1,chevrolet chevelle malibu
2,15,8,350,165,3693,11.5,70,1,buick skylark 320
3,18,8,318,150,3436,11.0,70,1,plymouth satellite
4,16,8,304,150,3433,12.0,70,1,amc rebel sst
5,17,8,302,140,3449,10.5,70,1,ford torino
6,15,8,429,198,4341,10.0,70,1,ford galaxie 500


In [4]:
#FIRST CHOICE OF VALIDATION SET: 196 RANDOM SAMPLES
set.seed(1)
train <- sample(392, 196)

In [5]:
#Linear regression
lm_fit <- lm(mpg ~ horsepower, subset = train)
#calculate the error on the validation set
#Note that the -train index below selects only the observations 
#that are not in the training set
mean((mpg - predict(lm_fit, Auto))[-train]^2)

[1] 23.26601

In [6]:
#Quadratic regression
lm_fit2 <- lm(mpg ~ poly(horsepower,2), subset = train)
mean((mpg - predict(lm_fit2, Auto))[-train]^2)

[1] 18.71646

In [7]:
#Cubic regression
lm_fit3 <- lm(mpg ~ poly(horsepower,3), subset = train)
mean((mpg - predict(lm_fit3, Auto))[-train]^2)

[1] 18.79401

The estimated test MSE for the linear regression fit is 23.27. For the quadratic and cubic regressions, the error rates are 18.72 and 18.79, respectively. Quadratic model seems a better fit than both linear and cubic models.  If we choose a different training set instead, then we will obtain somewhat different errors on the validation set.

In [8]:
# SECOND CHOICE OF VALIDATION SET: ANOTHER 196 RANDOM SAMPLES
set.seed(2)
train2 <- sample(392, 196)

In [9]:
#Linear regression
lm_fit <- lm(mpg ~ horsepower, subset = train2)
mean((mpg - predict(lm_fit, Auto))[-train2]^2)

#Quadratic regression
lm_fit2 <- lm(mpg ~ poly(horsepower,2), subset = train2)
mean((mpg - predict(lm_fit2, Auto))[-train2]^2)

#Cubic regression
lm_fit3 <- lm(mpg ~ poly(horsepower,3), subset = train2)
mean((mpg - predict(lm_fit3, Auto))[-train2]^2)

[1] 25.72651

[1] 20.43036

[1] 20.38533

Using the second validation set, the quadratic model still does better than the linear model, but is slightly worse than the cubic. 

Here are the findings from the using the 2 different validation sets above:  a model that predicts *mpg* using a quadratic function of *horsepower* performs better than a model that involves only a linear function of *horsepower*, and there is little evidence in favor of a model that uses a cubic function of *horsepower*.

# Leave-One-Out Cross Validation (LOOCV)

Like the validation set approach, LOOCV involves splitting the set of observations into two parts. 
- However, instead of creating two subsets of comparable size, a single observation $(x_1,y_1)$ is used for the validation set, and the remaining observations ${(x_2, y_2), \ldots , (x_n, y_n)}$ make up the training set. The statistical learning method is fit on the $n − 1$ training observations, and a prediction $\hat{y}_1$ is made for the excluded observation using its value $x_1$. Since $(x_1, y_1)$ was not used in the fitting process, $MSE_1 = (y_1 − \hat{y}_1)^2$ provides an approximately unbiased estimate for the test error. But even though $MSE_1$ is unbiased for the test error, it is a poor estimate because it is highly variable, since it is based upon a single observation $(x_1, y_1)$.

- We can repeat the procedure by selecting $(x_2,y_2)$ for the validation data, training the statistical learning procedure on the $n − 1$ observations ${(x_1,y_1),(x_3,y_3), \ldots,(x_n,y_n)}$,and computing $MSE_2 =(y_2−\hat{y}_2)^2$. 

- Repeating this approach n times produces n squared errors, $MSE_1, \ldots, MSE_n$. The LOOCV estimate for the test MSE is the average of these n test error estimates:

    - $CV_n = \frac{1}{n}\sum_{i=1}^n MSE_i$
    
- LOOCV has a couple of major advantages over the validation set approach. First, it has far less bias. In LOOCV, we repeatedly fit the statistical learning method using training sets that contain $n − 1$ observations, almost as many as are in the entire data set. This is in contrast to the validation set approach, in which the training set is typically around half the size of the original data set. 
  - Consequently, the LOOCV approach tends not to overestimate the test error rate as much as the validation set approach does.
  - Second, in contrast to the validation approach which will yield different results when applied repeatedly due to randomness in the training/validation set splits, performing LOOCV multiple times will always yield the same results: there is no randomness in the training/vali- dation set splits.


The LOOCV estimate can be automatically computed for any generalized linear model using the *glm()* and *cv.glm()* functions. The *glm()* function can be used to perform logistic regression by passing in the *family = "binomial"* argument. Here, we will perform linear regression using the *glm()* function rather than the *lm()* function because the former can be used together with *cv.glm()*. The *cv.glm()* function is part of the *boot* library.

In [10]:
library(boot)

In [11]:
glm_fit <- glm(mpg ~ horsepower)
cv_err <- cv.glm(Auto, glm_fit)
cv_err

$call
cv.glm(data = Auto, glmfit = glm_fit)

$K
[1] 392

$delta
[1] 24.23151 24.23114

$seed
  [1]       10403         292   241038711  -724551010  -982165160 -1828904773
  [7]  1219649113   639708648  -249104362 -1532738671  -205493181  1178813266
 [13]   334776292   619230759  -943223171   845950836   574669658  1135228325
 [19]  1324937663  -225314234  -259712816  1901458163   760603665  1517777472
 [25]  -353409506   333918441  1007243803  1823191594 -1096755892  2047555343
 [31]    36174405 -1271550980  -709741998  1365872589   667289991 -2093876114
 [37]  2061812808  1112530507  -120106071  -226796808   147205670   833570625
 [43]   791874771 -1221193598   282619572  -321910217 -1307274963  1192534596
 [49]  -516456598 -1233040427 -1928163985  1129724022  1863340960  -338181021
 [55]  -448311039  -256091024 -2016258034  -559437703  1778395147 -1105262278
 [61]   166402108   -19181697   426083541  1074993388  1545622082  1292039581
 [67]  1261006871  -194927234  -601294024 -189233

In [12]:
cv_err$delta

[1] 24.23151 24.23114

 The *cv.glm()* function produces a list with several components. The two numbers in the delta vector contain the cross-validation results. In this case the numbers are identical (up to two decimal places) and correspond to the LOOCV statistic.

## LOOCV for polynomial fits up to order 10

We can repeat this procedure for increasingly complex polynomial fits.
To automate the process, we use the *for()* function to initiate a for loop *for()*
which iteratively fits polynomial regressions for polynomials of order *i = 1* to *i = 10*, computes the associated cross-validation error, and stores it in the ith element of the vector *cv_error*.

In [13]:
#intialize the cv_err vector
cv_err <- rep(0,10)
#loop
for(i in 1:10){
    glm_fit <- glm(mpg ~ poly(horsepower,i))
    cv_err[i] <- cv.glm(Auto, glm_fit)$delta[1]
}

In [14]:
cv_err

[1] 24.23151 19.24821 19.33498 19.42443 19.03321 18.97864 18.83305 18.96115
 [9] 19.06863 19.49093

We see a sharp drop in the estimated test MSE between the linear and quadratic fits, but then no clear improvement from using higher-order polynomials.

# k-Fold Cross Validation

An alternative to LOOCV is $k$-fold CV. This approach involves randomly dividing the set of observations into $k$ groups, or folds, of approximately equal size. The first fold is treated as a validation set, and the method is fit on the remaining $k - 1$ folds. The mean squared error, $MSE_1$, is then computed on the observations in the held-out fold. This procedure is repeated k times; each time, a different group of observations is treated as a validation set. This process results in $k$ estimates of the test error, $MSE_1, MSE_2, \ldots, MSE_k$. The $k$-fold CV estimate is computed by averaging these values,

- $CV_k = \frac{1}{k}\sum^k_{i=1}MSE_i$

In [15]:
set.seed(17)
cv_err_10 <- rep(0,10)
for(i in 1:10){
    glm_fit <- glm(mpg ~ poly(horsepower, i))
    cv_err_10[i] <- cv.glm(Auto, glm_fit, K = 10)$delta[1]
}
cv_err_10

[1] 24.27207 19.26909 19.34805 19.29496 19.03198 18.89781 19.12061 19.14666
 [9] 18.87013 20.95520

# References

Chapter 5, **An Introduction to Statistical Learning
with Applications in R**, Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani